### 【 일변량 데이터기반 회귀 모델 】
- 주__제 : 학습 시간에 따른 수능 점수 예측 서비스
- 데이터 : study_score_easy.csv
- 구__성 : 피쳐(hour) + 타겟(score)
- 학__습 : 지도학습 + 회귀
- 구__현 : 인공신경망

[1] 모듈로딩 및 데이터 준비 <hr>

In [ ]:
#%pip install visdom

     ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
     ---------------------------------------- 1.4/1.4 MB 18.3 MB/s  0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408403 sha256=19dbc8d4e1ce3dad0583ce10b10c264bf84a27cb01d29397363a9632ecf93e3f
  Stored in directory: c:\users\kdt\appdata\local\pip\cache\wheels\42\29\49\5bed207bac4578e4d2c0c5fc0226bfd33a7e2953ea56356855
Successfully built visdom

   ------------------------------ --------- 3/4 [visdom]
   ---------------------------------------- 4/4 [visdom]

Note: you may need to restart the kernel to use updated packages.


In [10]:
## [1-1] 모듈 로딩
import pandas as pd                 # 데이터 분석 및 처리용 모듈
import torch                        # 텐서 및 수치, 기본 함수용 모듈
import torch.nn as nn               # 인공신경망 관련 모듈
import torch.nn.functional as F     # 인공신경망 함수(AF, LF, MF) 관련 모듈
from torchinfo import *

In [11]:
## [1-2] 데이터 관련
DATA_FILE = '../Data/study_score_easy.csv'

##- 데이터 로딩
dataDF = pd.read_csv(DATA_FILE)
dataDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   hours   1000 non-null   float64
 1   score   1000 non-null   float64
dtypes: float64(2)
memory usage: 15.8 KB


[2] ANN 모델 설계 <hr>

In [12]:
## -------------------------------------------------------------
##          입력수         퍼셉트론수/출력수         AF
## -------------------------------------------------------------
## 입력층       1개                 1개           ★ Pytorch에는
##                                               입력층 클래스 X
##                                               d입력 텐서를 입력층으로 간주
## 은닉층       1개                10개           ReLU
## 은닉층      10개                15개           ReLU
## 출력층      15개                 1개           -     회귀
## -------------------------------------------------------------
## 클래스이름 : ScoreModel
## 부모클래스 : nn.Module
## 오버라이딩 : __init__(self)   : 층 구성 요소 인스턴스 생성
##            forward(self, x) : 순전파 진행 메서드
##                               x -> 입력층으로 간주!
## -------------------------------------------------------------
class ScoreModel(nn.Module):
    ##- 층 구성 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()
        self.hd1_layer=nn.Linear(1, 10)
        self.hd2_layer=nn.Linear(10, 15)
        self.out_layer=nn.Linear(15, 1)

    ##- 순전파 진행 메서드
    def forward(self, x):
        ## 입력층 -> 은닉층 :  1 -> 10
        out = self.hd1_layer(x)
        out = F.relu(out)

        ## 은닉층 -> 은닉층 :  10 -> 15
        out = self.hd2_layer(out)
        out = F.relu(out)

        ## 은닉층 -> 은닉층 :  15 -> 1
        ## 회귀모델로 AF 없음!
        out = self.out_layer(out)

In [13]:
## 모델 인스턴스 생성
model  = ScoreModel()
summary(model, input_size=(1,1))

Layer (type:depth-idx)                   Output Shape              Param #
ScoreModel                               --                        --
├─Linear: 1-1                            [1, 10]                   20
├─Linear: 1-2                            [1, 15]                   165
├─Linear: 1-3                            [1, 1]                    16
Total params: 201
Trainable params: 201
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00